In [9]:
from pathlib import Path
import panel as pn
import pandas as pd
import numpy as np
pn.extension('plotly')
import hvplot.pandas
import matplotlib.pyplot as plt
import plotly.express as px
from panel.interact import interact, interactive, fixed, interact_manual

# Set the file paths for the XLSX using the Path class from the pathlib library
historical_excel_path = Path("../data/REQ-December-2021-Historical-Data.xlsx")
forecast_excel_path = Path("../data/REQ-December-2021-Forecast-Data.xlsx")

# Reading XLSX file to obtain production data
acp = pd.read_excel(historical_excel_path, sheet_name = 8, header = 6, usecols = "F:AM")
acp = acp.dropna()
acp = acp.rename(columns={'Unnamed: 5':'Commodity'})
acp.set_index(['Commodity'], inplace=True)
acp = acp.drop(columns="unit")

# Reading XLSX file to obtain exports data
ace = pd.read_excel(historical_excel_path, sheet_name = 3, header = 6, usecols = "F:AM")
ace = ace.dropna()
ace = ace.rename(columns={'Unnamed: 5':'Commodity'})
ace.set_index(['Commodity'], inplace=True)
ace = ace.drop(columns="unit")

# Utility functions to plot
def get_plot(df):
    return df.hvplot.line(x=df.columns, grid=True, responsive=True, height=300)

# Helper create_line_chart function
def create_bar_chart(data, title, xlabel, ylabel):
    """
    Create a bar chart based in the data argument.
    """
    return data.hvplot( title=title, xlabel=xlabel, ylabel=ylabel,rot=90)


In [10]:
'''
Read xlsx sheets containing production data state wise for:
    0* Aluminium:27, 4,kt
    1* Copper:30, 6, kt
    2* Diamonds:31,3 ,'000ct'
    3* Gold:32, 8, t
    4* Iron Ore:33, 5, kt
    5* Lead:34, 7, kt
    6* Nickel:35, 3,kt
    7* Silver:40, 7, t
    8* Tin :41, 3, t
    9* Uranium:42, 3, t
    10* Zinc:43, 7, kt
'''
meta_ = {'Aluminium':[27, 4,'kt'],
     'Copper':[30, 6, 'kt'],
     'Diamonds':[31,3 ,'000ct'],
     'Gold':[32, 8, 't'],
     'Iron Ore':[33, 5, 'kt'],
     'Lead':[34, 7, 'kt'],
     'Nickel':[35, 3,'kt'],
     'Silver':[40, 7, 't'],
     'Tin' :[41, 3, 't'],
     'Uranium':[42, 3, 't'],
     'Zinc':[43, 7, 'kt']
        }

sheets = [27,30,31,32,33,34,35,40,41,42,43]
plots = []
title = ''
commodities =  pd.read_excel(historical_excel_path, sheet_name = sheets,header = 7, usecols = "F:EF", parse_dates=True)
for sheet_name,sheet_num in meta_.items():
    commodities[sheet_num[0]] = commodities[sheet_num[0]].dropna()
    commodities[sheet_num[0]] = commodities[sheet_num[0]].drop(columns=["unit",'2019–20','2020–21'])
    commodities[sheet_num[0]] = commodities[sheet_num[0]].set_index(['Production'], inplace=False)
    commodities[sheet_num[0]] = commodities[sheet_num[0]].T
    commodities[sheet_num[0]] = commodities[sheet_num[0]].iloc[:,0:meta_[sheet_name][1]]
    if (meta_[sheet_name][2] == 'kt'):
        title =  sheet_name + ' Production in kilo tonnes'
    elif (meta_[sheet_name][2] == 't'):
        title =  sheet_name + ' Production in tonnes'
    elif (meta_[sheet_name][2] == '000ct'):
        title =  sheet_name + ' Production in carets'
    plots.append(commodities[sheet_num[0]].hvplot.bar(title=title,rot=90,height=400, width=800))


In [12]:
# Chart for Annual Commodity Production
acp_clean = acp.T
acp_clean = acp_clean.replace('na' ,np.nan)
acp_plt = acp_clean.hvplot.line(title="Annual Production of commodities in Australia",rot=90,height=400, width=800)


In [13]:
# Interactive Chart for Annual Commodity Production

def acp_plot(x):
    return acp_clean[x].hvplot.bar(rot=90)

interact(acp_plot, x=acp_clean.columns)


Column
    [0] Column
        [0] Select(name='x', options=['Black-saleable', ...], value='Black-saleable')
    [1] Row
        [0] HoloViews(Bars, name='interactive04984')

In [14]:
# Chart for Annual Commodity Exports
ace_clean = ace.T
ace_clean = ace_clean.replace('na',np.nan)
ace_plt = ace_clean.hvplot.bar(title='Annual Commodity Exports',rot=90,height=400, width=1000)


In [15]:
# Interactive Chart for Annual Commodity Exports
def ace_plot(x):
    return ace_clean[x].hvplot.bar(rot=90)
interact(ace_plot,x=ace_clean.columns)

Column
    [0] Column
        [0] Select(name='x', options=['Bauxite', 'Alumina', ...], value='Bauxite')
    [1] Row
        [0] HoloViews(Bars, name='interactive05237')

In [26]:
# Title
dash_title = pn.Row(
    pn.Column('## Analysis of commodities in Australia')
)

# Define a welcome text
desc = pn.panel("""This dashboard compares ** different commodities in Australia** using 

- Production by commodity, 
- Australia's commodity exports,and
- Commodity production by state """)

ticker = pn.widgets.Select(name='Commodities', options=[0,1,2,3,4,5,6,7,8,9,10])
#window = pn.widgets.IntSlider(name='Years', value=DateTime(1990-06-01), start=1, end=21)
row = pn.Row(
    pn.Column(title, ticker),
    plots[ticker.options[0]]
)
def update(event):
    row[1].object = plots[ticker.value]

ticker.param.watch(update, 'value')


# Create the main dashboard
Australia_commodity_dashboard = pn.Tabs(
    (
        "Welcome",
        desc
    ),
    (
        "Production by commodity",
        acp_plt
    ),
    (
        "Australia's commodity exports",
        ace_plt
    ),
    (
        "Commodity production by state",
        row
    )
)


'\n'

In [28]:
dashboard = pn.Column(dash_title,Australia_commodity_dashboard )
dashboard.servable()

Column
    [0] Row
        [0] Column
            [0] Markdown(str)
    [1] Tabs
        [0] Markdown(str)
        [1] HoloViews(NdOverlay, name='Production by commodity')
        [2] HoloViews(Bars, name="Australia's c...)
        [3] Row
            [0] Column
                [0] Markdown(str)
                [1] Select(name='Commodities', options=[0, 1, 2, 3, 4, ...], value=0)
            [1] HoloViews(Bars)